# Setup

In [1]:
!nvidia-smi

Wed Mar 26 01:26:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P5              5W /   25W |     639MiB /   6141MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip3 install -r requirements.txt -q

In [3]:
from tensorflow.keras import backend as K
print(K.backend())

tensorflow


In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
import imgaug
import time
from memory_profiler import memory_usage

try:
    from tensorflow.keras.callbacks import Callback  # TensorFlow < 2.14
except ImportError:
    from keras.callbacks import Callback  # TensorFlow 2.14+

from tqdm import tqdm
import skimage.io
import tensorflow as tf
import h5py
from tensorflow.python.keras.saving import hdf5_format
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import warnings

In [6]:
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", category=UserWarning)

# Model

## Config

In [7]:
# Root directory of the project
ROOT_DIR = os.getcwd()
DATASET_DIR = os.path.join(ROOT_DIR, "dataset")

In [8]:
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

In [9]:
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

class CustomConfig(Config):
    """Configuration for training on the poultry feces dataset."""
    NAME = "class"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 3

    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 512

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # Background + (cocci, healthy, ncd, salmo)

    STEPS_PER_EPOCH = 452 / IMAGES_PER_GPU
    VALIDATION_STEPS = 100 / IMAGES_PER_GPU
    # STEPS_PER_EPOCH = 100
    # VALIDATION_STEPS = 50

    DETECTION_MIN_CONFIDENCE = 0.9
    LEARNING_RATE = 0.001

In [10]:
config = CustomConfig()

##  Dataset

In [11]:
CLASS_NAMES = ["BG", "cocci", "healthy", "ncd", "salmo"] 

In [12]:
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):
        """Load a subset of the poultry feces dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("class", 1, "cocci")
        self.add_class("class", 2, "healthy")
        self.add_class("class", 3, "ncd")
        self.add_class("class", 4, "salmo")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        json_file = os.path.join(dataset_dir, f"{subset}_annotations.json")
        with open(json_file) as f:
          annotations1 = json.load(f)

        # print(annotations1)
        annotations = list(annotations1.values())
        annotations = [a for a in annotations if a['regions']]

        # Add images
        name_dict = {"cocci": 1, "healthy": 2, "ncd": 3, "salmo": 4}
        # for a in annotations:
        for a in tqdm(annotations, desc="Loading Images ..."):
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']]
            classes = [s['region_attributes']['class'] for s in a['regions']]

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in classes]

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "class",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image."""
        image_info = self.image_info[image_id]
        if image_info["source"] != "class":
            return super(self.__class__, self).load_mask(image_id)

        if "polygons" not in image_info:
            print(f"Skipping image {image_info['path']} - No polygons found.")
            return np.zeros((image_info["height"], image_info["width"], 0), dtype=np.uint8), np.array([])

        num_ids = image_info['num_ids']
        mask = np.zeros([image_info["height"], image_info["width"], len(image_info["polygons"])], dtype=np.uint8)

        for i, p in enumerate(image_info["polygons"]):
            if "all_points_x" in p and "all_points_y" in p:  # Polygon Annotation
                rr, cc = skimage.draw.polygon(np.array(p['all_points_y']), np.array(p['all_points_x']))
            elif "name" in p and p["name"] == "ellipse":  # Ellipse Annotation
                cy, cx, ry, rx = p["cy"], p["cx"], p["ry"], p["rx"]
                rr, cc = skimage.draw.ellipse(cy, cx, ry, rx, shape=(image_info["height"], image_info["width"]))
            else:
                print(f"Skipping annotation with missing keys: {p}")
                continue  # Skip this polygon if missing required points

            # Clip to prevent out-of-bounds errors
            rr = np.clip(rr, 0, image_info["height"] - 1)
            cc = np.clip(cc, 0, image_info["width"] - 1)

            mask[rr, cc, i] = 1  # Assign mask values

        return mask.astype(bool), np.array(num_ids, dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "class":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [13]:
dataset_train = CustomDataset()
dataset_train.load_custom(DATASET_DIR, "train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_custom(DATASET_DIR, "val")
dataset_val.prepare()

Loading Images ...: 100%|██████████| 240/240 [00:08<00:00, 27.87it/s]


## Train

In [14]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_poultry.h5")

In [15]:
def train(model, dataset_train, dataset_val):
    """Train the model."""
    start_time = time.time()
    model.train(
            dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=100,
            layers="heads",
            augmentation=imgaug.augmenters.Sequential([
                imgaug.augmenters.Grayscale(alpha=(0.0, 1.0)),
                imgaug.augmenters.AddToHueAndSaturation((-20, 20)),
                imgaug.augmenters.Add((-10, 10), per_channel=0.5),
                imgaug.augmenters.Invert(0.05, per_channel=True),
                imgaug.augmenters.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))
            ])
        )
    model.keras_model.save_weights(WEIGHTS_PATH)
    print(f"\nModel saved at {WEIGHTS_PATH}\n")
    end_time = time.time()

    # Output training time and memory usage statistics
    print(f"Training time: {end_time - start_time:.2f} seconds")

In [16]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

if os.path.exists(COCO_WEIGHTS_PATH):
    print("COCO weights found.\n")
    model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
elif os.path.exists(WEIGHTS_PATH):
    print("Poultry weights found.\n")
    model.load_weights(WEIGHTS_PATH, by_name=True)
else:
    print("No weights found. Training from scratch.\n")

train(model, dataset_train, dataset_val)

COCO weights found.


Starting at epoch 0. LR=0.001

Checkpoint Path: c:\Users\Artryan Nero Logarta\OneDrive - Mapúa University\24-25\2Q\MCS612\Automated-Poultry-Disease-Detection-from-Fecal-Images-Using-Mask-R-CNN\logs\class20250326T0127\mask_rcnn_class_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

## Evaluate

In [17]:
class InferenceConfig(Config):
    NAME = "class-labels"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 512
    NUM_CLASSES = 1 + 4  # Background + 4 disease classes
    DETECTION_MIN_CONFIDENCE = 0.1  # Lower to include more predictions

config = InferenceConfig()

In [18]:
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=ROOT_DIR)
model.load_weights(WEIGHTS_PATH, by_name=True)

In [19]:
def evaluate_model(model, dataset, config):
    """Evaluate the Mask R-CNN model and print various metrics."""

    APs = []
    gt_all = []
    pred_all = []
    with tqdm(dataset.image_ids, desc="Evaluating images", leave=True) as pbar:
        for image_id in pbar:
            # Load image and ground truth data
            image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(
                dataset, config, image_id
            )

            # Run object detection
            results = model.detect([image], verbose=0)
            r = results[0]

            # Compute AP
            AP, precisions, recalls, _, iou_threshold, conf = utils.compute_ap(
                gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r["masks"]
            )
            APs.append(AP)

            # Store ground truth and predicted class IDs
            gt_labels = gt_class_id.tolist()
            pred_labels = r["class_ids"].tolist()

            # Ensure lengths match
            max_len = max(len(gt_labels), len(pred_labels))

            # Pad with background class (0) where necessary
            gt_labels += [0] * (max_len - len(gt_labels))
            pred_labels += [0] * (max_len - len(pred_labels))

            gt_all.extend(gt_labels)
            pred_all.extend(pred_labels)

            # Update the progress bar with current image stats
            prec = precisions[-2]
            rec = recalls[-2]
            conf = np.array(conf)
            conf_str = "[" + "][".join(", ".join(map(str, row)) for row in conf) + "]"

            pbar.set_postfix(
                image_id=image_id, 
                AP=f"{AP:.3f}", 
                Precision=f"{prec:.3f}", 
                Recall=f"{rec:.3f}", 
                ConfMatrix=f"{conf_str}"
            )
            
        

    print(f"\n📊 Summary Results:")
    print(f"\n--------------------------------------------")
    # Compute Mean Average Precision (mAP)
    mean_ap = np.mean(APs)
    print(f"\nMean Average Precision (mAP): {mean_ap:.4f}")

    # Compute Confusion Matrix
    cm = confusion_matrix(gt_all, pred_all, labels=[0, 1, 2, 3, 4, 5])
    print("\nFinal Confusion Matrix:\n", cm)

    # Compute Accuracy
    accuracy = accuracy_score(gt_all, pred_all)
    print(f"\nOverall Accuracy: {accuracy:.4f}")

    # Compute Precision, Recall, and F1-score
    print("\nClassification Report:")
    print(classification_report(gt_all, pred_all, labels=[1, 2, 3, 4], target_names=CLASS_NAMES[1:]))

    return mean_ap, cm, accuracy

# Run evaluation
_, _, _ = evaluate_model(model, dataset_val, config)


Evaluating images: 100%|██████████| 240/240 [01:18<00:00,  3.04it/s, AP=1.000, ConfMatrix=[1], Precision=1.000, Recall=1.000, image_id=239]       


📊 Summary Results:

--------------------------------------------

Mean Average Precision (mAP): 0.8356

Final Confusion Matrix:
 [[  0  20  23 118  55   0]
 [  5  59   0   3   1   0]
 [  0   0  54   8   3   0]
 [  2   0   4  58   1   0]
 [  0   0   0   7  53   0]
 [  0   0   0   0   0   0]]

Overall Accuracy: 0.4726

Classification Report:
              precision    recall  f1-score   support

       cocci       0.75      0.87      0.80        68
     healthy       0.67      0.83      0.74        65
         ncd       0.30      0.89      0.45        65
       salmo       0.47      0.88      0.61        60

   micro avg       0.48      0.87      0.62       258
   macro avg       0.55      0.87      0.65       258
weighted avg       0.55      0.87      0.65       258

